In [48]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [76]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary'])
]


In [77]:
print("General files:")
for f in general_files:
    print(f)
len(general_files)

General files:
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__apache__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__cochise__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__coconino__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__graham__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__greenlee__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__la_paz__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__maricopa__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__mohave__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__navajo__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\unfiltered\20161108__az__general__pima__precinct.csv
C:\Huy Phan\College\

14

In [51]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__apache__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__cochise__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__greenlee__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__la_paz__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__mohave__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__navajo_precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__pima__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__pinal__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\20160322__az__primary__president__yavapai_precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\cleaned\

In [52]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        
        # CHOOSING PRESIDENT ONLY
        # office, 
        # Combine precinct as county + precinct

        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"].str.upper().str.contains("\w*PRESIDENT\w*", na=False)]

        # cleaning precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]        
            df = df[df['precinct'].str.upper() != "ELECTION TOTAL"]     

        df["precinct"] = df["precinct"].astype(str).str.replace(r'^="|"$', '', regex=True)
        df["precinct"] = df["county"].astype(str).str.strip() + df["precinct"].astype(str).str.strip()
        df["precinct"] = df["precinct"].str.upper()
    

        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
pri_combined_df


,Unnamed: 0,contest,precinct,contest_id,office,choice,candidate,vote,county,precinct_id,...,party_name,precinct_name,precinct_designation,record_type,precinct_place,vote_for_value,contest_order_id,candidate_id,vote_type,done
0,2.0,20,APACHEALPINE,10.0,President - Dem,999999.0,Number of Precincts for Race,1,APACHE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,20,APACHEALPINE,10.0,President - Dem,999998.0,Number of Precincts Reporting,1,APACHE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,20,APACHEALPINE,10.0,President - Dem,999997.0,Registered Voters,80,APACHE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,20,APACHEALPINE,10.0,President - Dem,999996.0,Times Counted,41,APACHE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,20,APACHEALPINE,10.0,President - Dem,10.0,"De La Fuente, Roque",0,APACHE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12103,67.0,20,YUMA2,60.0,President - Grn CD4,999998.0,Number of Precincts Reporting,1,YUMA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12104,68.0,20,YUMA2,60.0,President - Grn CD4,999997.0,Registered Voters,7,YUMA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12105,69.0,20,YUMA2,60.0,President - Grn CD4,999996.0,Times Counted,0,YUMA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12106,70.0,20,YUMA2,60.0,President - Grn CD4,10.0,"Mesplay, Kent",0,YUMA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# cleaning 

In [54]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "vote"]]

primary_data

,precinct,party,candidate,vote
0,APACHEALPINE,NaN,Number of Precincts for Race,1
1,APACHEALPINE,NaN,Number of Precincts Reporting,1
2,APACHEALPINE,NaN,Registered Voters,80
3,APACHEALPINE,NaN,Times Counted,41
4,APACHEALPINE,NaN,"De La Fuente, Roque",0
...,...,...,...,...
12103,YUMA2,NaN,Number of Precincts Reporting,1
12104,YUMA2,NaN,Registered Voters,7
12105,YUMA2,NaN,Times Counted,0
12106,YUMA2,NaN,"Mesplay, Kent",0


In [55]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
UNDER VOTES                      1074
OVER VOTES                       1074
CRUZ, TED                         309
SANTORUM, RICK                    309
COOK, TIM                         309
PAUL, RAND                        309
RUBIO, MARCO                      309
GRAHAM, LINDSEY                   309
KASICH, JOHN R.                   309
BUSH, JEB                         309
TRUMP, DONALD J.                  309
FIORINA, CARLY                    309
PATAKI, GEORGE E.                 309
HUCKABEE, MIKE                    309
CHRISTIE, CHRIS                   309
CARSON, BEN                       309
STEINBERG, MICHAEL                309
SANDERS, BERNIE                   309
O'MALLEY, MARTIN J.               309
CLINTON, HILLARY                  309
HEWES, HENRY                      309
MESPLAY, KENT                     309
STEIN, JILL                       309
DE LA FUENTE, ROQUE "ROCKY"       301
Number of Precincts Reporting     162
Number of Precincts for Race      162
Re

In [56]:
# Cleaning Candidates

# Turning all primary data to uppercase
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"\w*NUMBER\w*",
    r"\w*TIME\w*",
]

pattern = "|".join(unwanted_keywords)

# Assuming candidate column is already string and uppercase
primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# Fixing the , candidate
primary_data["candidate"] = (
    primary_data["candidate"].str.split(",")
    .str[0]
    .str.strip()
    )

# Fixing De la Fuente
primary_data.loc[
    primary_data["candidate"].str.contains("\w*FUEN\w*", case=False, na=False),
    "candidate"
] = "FUENTE"

#Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# Fixing Huckabee
primary_data.loc[
    primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
    "candidate"
] = "HUCKABEE"

# Fixing Fiorina
primary_data.loc[
    primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
    "candidate"
] = "FIORINA"

# Fixing Gray
primary_data.loc[
    primary_data["candidate"].str.contains("GREY", case=False, na=False),
    "candidate"
] = "GRAY"

# Fixing Wilson
primary_data.loc[
    primary_data["candidate"].str.contains("WISON", case=False, na=False),
    "candidate"
] = "WILSON"

# Fixing the , candidate
primary_data["candidate"] = (
    primary_data["candidate"].str.split(",")
    .str[0]
    .str.strip()
    )
# Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\2623341596.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\2623341596.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = (
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\2623341596.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

candidate
FUENTE       420
STEINBERG    420
MESPLAY      420
CARSON       420
CHRISTIE     420
HUCKABEE     420
CRUZ         420
PATAKI       420
FIORINA      420
TRUMP        420
BUSH         420
KASICH       420
GRAHAM       420
RUBIO        420
PAUL         420
COOK         420
SANTORUM     420
HEWES        420
CLINTON      420
O'MALLEY     420
SANDERS      420
STEIN        420
Name: count, dtype: int64

In [57]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
REP                 5012
DEM                 2148
NaN                 1188
GRN                  716
REPUBLICAN PARTY     112
DEMOCRATIC PARTY      48
GREEN PARTY           16
Name: count, dtype: int64

In [58]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to fill remaining missing party from master lookup
#=====================================
def fill_party_from_master(df, master_df):
    party_map = master_df.set_index("candidate")["party"].to_dict()
    df["party"] = df.apply(
        lambda row: party_map.get(row["candidate"], row["party"])
        if pd.isna(row["party"]) else row["party"],
        axis=1
    )
    return df


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)
    
    # Combine unique pairs
    new_data = df[["candidate", "party"]].dropna().drop_duplicates()
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)

# Fill remaining party using general master CSV
# master_party_df = pd.read_csv("data/master_primary_candidate_party.csv")
# general_data = fill_party_from_master(general_data, master_party_df)

# STEP 3: Update master file with new (candidate, party) pairs
# update_master_candidate_party(general_data, "data/master_candidate_party.csv")
primary_data["party"].value_counts(dropna=False)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\509361827.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(


party
REP                 5768
DEM                 2472
GRN                  824
REPUBLICAN PARTY     112
DEMOCRATIC PARTY      48
GREEN PARTY           16
Name: count, dtype: int64

In [59]:
# Cleaning Party
primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)
primary_data.loc[:,"party"] = (
    primary_data["party"]
    .replace({
        "REPUBLICAN PARTY": "REP",
        "R": "REP",
        "Republican":"REP",
        "DEMOCRATIC PARTY": "DEM",
        "D": "DEM",
        "Democratic": "DEM",
        "GREEN PARTY":"GRN"
    })
    .fillna("IND")
)
primary_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\135971507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(


party
REP    5880
DEM    2520
GRN     840
Name: count, dtype: int64

In [60]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="vote",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\3570271567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_FUENTE,pri_dem_HEWES,pri_dem_O'MALLEY,pri_dem_SANDERS,pri_dem_STEINBERG,pri_grn_MESPLAY,pri_grn_STEIN,pri_rep_BUSH,...,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_GRAHAM,pri_rep_HUCKABEE,pri_rep_KASICH,pri_rep_PATAKI,pri_rep_PAUL,pri_rep_RUBIO,pri_rep_SANTORUM,pri_rep_TRUMP
0,APACHEALPINE,15,0,0,2,20,1,0,0,0,...,37,0,0,0,13,0,0,2,0,55
1,APACHECANYON,351,3,4,4,143,9,0,0,2,...,22,0,1,0,0,0,3,5,0,13
2,APACHECHINLE,178,0,0,5,79,1,0,0,4,...,12,0,0,0,7,0,0,2,1,13
3,APACHECONCHO,42,3,1,2,73,1,0,4,0,...,157,1,0,0,16,0,0,8,0,285
4,APACHECORNFIELDS,129,0,2,2,53,4,0,0,0,...,5,0,0,1,2,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,PINALWALKER BUTTE,108,0,0,1,101,2,0,0,0,...,138,1,0,0,19,0,0,27,0,206
407,YAVAPAI201,7187,69,52,140,6265,78,6,25,202,...,10809,58,31,89,3109,13,115,2995,26,16557
408,YAVAPAI217,1667,7,11,32,1579,10,3,9,29,...,1159,15,6,9,484,4,12,391,1,1961
409,YUMA1,3773,92,48,58,1908,29,1,0,54,...,1555,7,5,14,334,1,14,505,6,1966


In [61]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]           

        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

             
        
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
gen_combined_df

,county,precinct,votes,election_day,early_voting,provisional,polling_place_votes_ds200,early_votes_ds200,party,office,district,candidate,Unnamed: 0.1,Unnamed: 0,central_count,late_early_voting
0,Apache,APACHEALPINE,60,10,5,0.0,0.0,0.0,DEM,President,NaN,"CLINTON, KAINE",NaN,NaN,NaN,NaN
1,Apache,APACHECANYON DE CHELLY,1387,674,63,7.0,0.0,0.0,DEM,President,NaN,"CLINTON, KAINE",NaN,NaN,NaN,NaN
2,Apache,APACHECHINLE,678,300,34,3.0,0.0,0.0,DEM,President,NaN,"CLINTON, KAINE",NaN,NaN,NaN,NaN
3,Apache,APACHECONCHO,228,40,18,0.0,0.0,0.0,DEM,President,NaN,"CLINTON, KAINE",NaN,NaN,NaN,NaN
4,Apache,APACHECORNFIELDS,349,262,8,0.0,0.0,0.0,DEM,President,NaN,"CLINTON, KAINE",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4328,Yuma,YUMA44,0,0,0,0.0,NaN,NaN,REP,President,NaN,TRUMP / PENCE,NaN,NaN,NaN,0.0
4329,Yuma,YUMA44,0,0,0,0.0,NaN,NaN,DEM,President,NaN,CLINTON / KAINE,NaN,NaN,NaN,0.0
4330,Yuma,YUMA44,0,0,0,0.0,NaN,NaN,LIB,President,NaN,JOHNSON / WELD,NaN,NaN,NaN,0.0
4331,Yuma,YUMA44,0,0,0,0.0,NaN,NaN,GRN,President,NaN,STEIN / BARAKA,NaN,NaN,NaN,0.0


In [62]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,APACHEALPINE,DEM,"CLINTON, KAINE",60
1,APACHECANYON DE CHELLY,DEM,"CLINTON, KAINE",1387
2,APACHECHINLE,DEM,"CLINTON, KAINE",678
3,APACHECONCHO,DEM,"CLINTON, KAINE",228
4,APACHECORNFIELDS,DEM,"CLINTON, KAINE",349
...,...,...,...,...
4328,YUMA44,REP,TRUMP / PENCE,0
4329,YUMA44,DEM,CLINTON / KAINE,0
4330,YUMA44,LIB,JOHNSON / WELD,0
4331,YUMA44,GRN,STEIN / BARAKA,0


In [63]:
# Viewing candidate data
general_data["candidate"].value_counts(dropna=False)

candidate
WRITE-IN                 489
OVER VOTES               489
UNDER VOTES              489
STEIN                    350
JOHNSON                  350
TRUMP                    350
CLINTON                  350
CLINTON, KAINE           139
TRUMP, PENCE             139
JOHNSON, WELD            139
STEIN, BARAKA            139
STEIN / BARAKA           100
JOHNSON / WELD           100
CLINTON / KAINE          100
TRUMP / PENCE            100
Number of Under Votes     55
Times Over Voted          55
Times Blank Voted         55
Registered Voters         55
WRITE-IN 900              45
Overvote                  45
Undervote                 45
TotalVotes                45
Ballots Cast              44
Write-in 50               44
Write-in                  11
Times Counted             11
Name: count, dtype: int64

In [64]:
# Cleaning Candidates

# Turning all primary data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"\w*WRITE[\s-]\w*",
    r"NAN",
    r"UNCERTIFIED",
    r"UNVERIFIED",
    r"NONE OF THE ABOVE",
    r"LBT",
    r"DEM",
    r"REP",
    r"GRN",
    r"NAN",
    r"\w*CAST\w*",
    r"\w*COUNT\w*"

]

pattern = "|".join(unwanted_keywords)

# FILTER OUT TRASH WORDS
general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# Fixing Darrell Castle"
general_data.loc[
    general_data["candidate"].str.contains("\w*KAIN\w*", case=False, na=False),
    "candidate"
] = "CLINTON"

general_data.loc[
    general_data["candidate"].str.contains("\w*PENCE\w*", case=False, na=False),
    "candidate"
] = "TRUMP"

# Fixing mixed president + vice_president by /
general_data["candidate"] = (
    general_data["candidate"].str.split("/")
    .str[0]
    .str.strip()
    )

#Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# Fixing the , candidate
general_data["candidate"] = (
    general_data["candidate"].str.split(",")
    .str[0]
    .str.strip()
    )



# Fixing Cubbler
general_data.loc[
    ( 
        general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
        general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
        general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
    "candidate")
] = "CUBBLER"


# Fixing Kotlikoff
general_data.loc[
    general_data["candidate"].str.contains("\w*BARAKA\w*", case=False, na=False),
    "candidate"
] = "STEIN"

# Fixing Valdivia
general_data.loc[
    general_data["candidate"].str.contains("\w*WELD\W*", case=False, na=False),
    "candidate"
] = "JOHNSON"

# Fixing HOEFLING
general_data.loc[
    general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
    "candidate"
] = "HOEFLING"


# Fixing McMullin
general_data.loc[
    general_data["candidate"].str.contains("MCMULLEN", case=False, na=False) ,
    "candidate"
] = "MCMULLIN"

general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\3866137965.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\3866137965.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = (
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\3866137965.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

candidate
CLINTON    589
TRUMP      589
JOHNSON    589
STEIN      589
Name: count, dtype: int64

In [65]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
DEM    589
REP    589
GRN    589
LBT    534
LIB     55
Name: count, dtype: int64

In [66]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to fill remaining missing party from master lookup
#=====================================
def fill_party_from_master(df, master_df):
    party_map = master_df.set_index("candidate")["party"].to_dict()
    df["party"] = df.apply(
        lambda row: party_map.get(row["candidate"], row["party"])
        if pd.isna(row["party"]) else row["party"],
        axis=1
    )
    return df


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)
    
    # Combine unique pairs
    new_data = df[["candidate", "party"]].dropna().drop_duplicates()
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
general_data["party"] = general_data.apply(
    lambda row: fill_party_from_data(row, general_data),
    axis=1
)

# Fill remaining party using general master CSV
# master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\master_candidate_party.csv") # USE YOUR OWN ADDRESS
#  general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\1869924615.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = general_data.apply(


party
DEM    589
REP    589
GRN    589
LBT    534
LIB     55
Name: count, dtype: int64

In [67]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK"
    })
)

general_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\477868490.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = general_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\477868490.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = (


party
DEM    589
REP    589
LIB    589
GRN    589
Name: count, dtype: int64

In [68]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [69]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29772\1829951435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate_column"] = (


candidate_column,precinct,gen_dem_CLINTON,gen_grn_STEIN,gen_lib_JOHNSON,gen_rep_TRUMP
0,APACHEALPINE,60,2,13,216
1,APACHECANYON DE CHELLY,1387,32,72,143
2,APACHECHINLE,678,17,38,93
3,APACHECONCHO,228,11,44,1135
4,APACHECORNFIELDS,349,3,16,33
...,...,...,...,...,...
584,YUMA5,461,4,58,808
585,YUMA6,902,24,47,298
586,YUMA7,1351,36,39,199
587,YUMA8,267,5,24,515


In [70]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Save combined
combined.to_csv("TX.csv", index=False)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts




In [71]:

combined.to_csv("AZ.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("AZ_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("AZ_general_filtered.csv", index=False)

In [72]:
print(f"primary: {len(primary_precincts)}, general: {len(general_precincts)}, combined: {len(combined)}")

primary: 411, general: 589, combined: 388


In [73]:
print(f"primary: {len(pri_combined_df['county'].unique())}, general: {len(gen_combined_df['county'].unique())}")


primary: 10, general: 10


In [74]:
pri_combined_df['county'].unique()

array(['APACHE', 'COCHISE', 'GREENLEE', 'LA PAZ', 'MOHAVE', 'NAVAJO',
       'PIMA', 'PINAL', 'YAVAPAI', 'YUMA'], dtype=object)

In [75]:
gen_combined_df['county'].unique()


array(['Apache', 'Cochise', 'Greenlee', 'La Paz', 'Mohave', 'Navajo',
       'Pima', 'Pinal', 'Yavapai', 'Yuma'], dtype=object)